<a href="https://colab.research.google.com/github/SohailVibeCoder/Olist-ML-project/blob/main/FoF_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <u>**TASK 1**</u>

Question 1



In [1]:
#FAIR PRICE OF BOND K
# C=5% of £1000 = £50
CFfivetofourteen= sum(50 / (1.08**t) for t in range(5, 15))


CFone=50/1.04
CFtwo=50/1.05**2
CFthree=50/1.06**3
CFfour=50/1.07**4
CFfifteen=1050/1.08**15

priceK=round(CFone+CFtwo+CFthree+CFfour+CFfivetofourteen+CFfifteen,2)
print(f"Fair price of bond K is {priceK} GBP")

#FAIR PRICE OF BOND L
PriceL=round(500/(1.1)**15,3)
print(f"Fair price of bond L is {PriceL} GBP")

#FAIR PRICE OF BOND K
# C=20% of £2000 = £400
CFfivetonineteen= sum(400 / (1.2**t) for t in range(5, 20))

CFoneM=400/1.1
CFtwoM=400/1.12**2
CFthreeM=400/1.14**3
CFfourM=400/1.16**4
CFtwenty=2400/1.2**20

priceM=round(CFoneM+CFtwoM+CFthreeM+CFfourM+CFfivetonineteen+CFtwenty,2)
print(f"Fair price of bond M is {priceM} GBP")

Fair price of bond K is 751.16 GBP
Fair price of bond L is 119.696 GBP
Fair price of bond M is 2137.93 GBP


Question 3

In [2]:
!pip install numpy_financial

In [3]:

import numpy_financial as npf

#YTM FOR BOND K

YTMK=round(npf.rate(15,50,-751.16,1000),4)
print(f"YTM of bond K is {100*YTMK}%")

#YTM FOR BOND L

YTML=round((500/119.7)**(1/15)-1,4)
print(f"YTM of bond L is {100*YTML}%")

#YTM FOR BOND M
YTMM=round(npf.rate(20,400,-2137.93,2000),4)
print(f"YTM of bond M is {100*YTMM}%")

YTM of bond K is 7.89%
YTM of bond L is 10.0%
YTM of bond M is 18.67%


Question 4

Task 2

Question 1

In [5]:
from google.colab import drive
import os
import pandas as pd
import numpy as np

drive.mount('/content/drive')

os.chdir("/content/drive/MyDrive/")

crsp_monthly=pd.read_csv("crsp_monthly_subsample.csv")


# Convert date to datetime format
crsp_monthly['date'] = pd.to_datetime(crsp_monthly['date'])

# Clean price and return columns
crsp_monthly['prc'] = crsp_monthly['prc'].abs()
crsp_monthly['ret'] = pd.to_numeric(crsp_monthly['ret'], errors='coerce')
crsp_monthly.drop_duplicates(subset=['date', 'tsymbol', 'prc','ret','vol','shrout','comnam'],inplace=True
)

crsp_monthly.to_csv("crsp_monthly.csv", index=False)
crsp_monthly


ff5_data = pd.read_csv("F-F_Research_Data_5_Factors_2x3.csv", skiprows=3, nrows=748)
# Rename first column to 'date'
ff5_data.rename(columns={ff5_data.columns[0]: 'date',
                         ff5_data.columns[1]: 'mktrf'}, inplace=True)
# Make all other columns lowercase
ff5_data.columns = [col.lower() for col in ff5_data.columns]
# Convert date to datetime format assuming that the data is valid as of the end of the month
ff5_data['date'] = pd.to_datetime(ff5_data['date'], format='%Y%m') + pd.offsets.MonthEnd(0)
ff5_data.to_csv("ff5_data.csv", index=False)




# Import Momentum factor returns data
# Note: careful with skiprows and nrows parameters to get the correct data
# - inspect the CSV file to determine these values
mom_data = pd.read_csv( "F-F_Momentum_Factor.csv", skiprows=13, nrows=1186)
mom_data.rename(columns={mom_data.columns[0]: 'date',
                         mom_data.columns[1]: 'mom'}, inplace=True)
# Convert date to datetime format assuming that the data is valid as of the end of the month
mom_data['date'] = pd.to_datetime(mom_data['date'], format='%Y%m') + pd.offsets.MonthEnd(0)
mom_data.to_csv("mom_data.csv", index=False)

Mounted at /content/drive


In [6]:
# Merge FF5 and Momentum data
factor_data = pd.merge(ff5_data, mom_data, on='date', how='left')
factor_data.to_csv("factor_data.csv", index=False)
# Since FF5 data are in percentages, convert them to decimals
factor_data[['mktrf', 'smb', 'hml', 'rmw', 'cma', 'mom', 'rf']] = factor_data[['mktrf', 'smb', 'hml', 'rmw', 'cma','mom', 'rf']] / 100

In [7]:
start_date = '2015-01-01'
end_date = '2024-12-31'

date=(factor_data['date'] >= start_date) & (factor_data['date'] <= end_date)
factor_data=factor_data[date]
factor_data.to_csv("factor_data_filtered.csv",index=False)

In [8]:
factor_cols = ['mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'mom']
avgreturns=factor_data[factor_cols].mean()
print("Table B1:")
print(avgreturns)

Table B1:
mktrf    0.009820
smb     -0.001664
hml     -0.001841
rmw      0.003903
cma     -0.000866
rf       0.001395
mom      0.002009
dtype: float64


Question 2

In [9]:
avgreturns_stock = crsp_monthly.groupby('permno')['ret'].mean()
print("Table B2:")
print(avgreturns_stock)

Table B2:
permno
10026    0.006283
10032    0.014102
10044   -0.006478
10065    0.012091
10104    0.014750
10107    0.021769
10318    0.010662
10333    0.017095
12079    0.010284
21020   -0.001801
Name: ret, dtype: float64


Question 3

In [10]:
# Merge on year and month
# - convert date to yearmonth variable
crsp_monthly['yearmonth'] = crsp_monthly['date'].dt.to_period('M')
factor_data['yearmonth'] = factor_data['date'].dt.to_period('M')

# - perform the merge
crsp_factors = pd.merge(crsp_monthly, factor_data.drop(columns=['date']), on='yearmonth', how='left')
crsp_factors.to_csv("crsp_factors.csv", index=False)

# Compute excess returns for individual stocks
# - denote excess returns as 'retex'
crsp_factors['retex'] = crsp_factors['ret'] - crsp_factors['rf']

avgexcessreturns_stock = crsp_factors.groupby('permno')['retex'].mean()
print("Table B3:")
print(avgexcessreturns_stock)

Table B3:
permno
10026    0.004888
10032    0.012707
10044   -0.007873
10065    0.010696
10104    0.013355
10107    0.020374
10318    0.009267
10333    0.015700
12079    0.008889
21020   -0.003196
Name: retex, dtype: float64


Question 4

In [11]:
return_stats = crsp_factors.groupby('permno').agg(
    mean_ret=('ret', 'mean'),
    mean_retex=('retex', 'mean'),
    std_ret=('ret', 'std'),
    std_retex=('retex', 'std')
)


In [12]:
# Compute monthly Sharpe ratios of individual stocks
return_stats['sharpe_ratio_MON'] = return_stats['mean_retex'] / return_stats['std_retex']
# Compute annualised Sharpe ratios
return_stats['sharpe_ratio_ANN'] = return_stats['sharpe_ratio_MON'] * np.sqrt(12)

return_stats


,mean_ret,mean_retex,std_ret,std_retex,sharpe_ratio_MON,sharpe_ratio_ANN
permno,,,,,,
10026,0.006283,0.004888,0.063984,0.064001,0.076370,0.264554
10032,0.014102,0.012707,0.076791,0.076742,0.165579,0.573583
10044,-0.006478,-0.007873,0.107124,0.107379,-0.073324,-0.254001
10065,0.012091,0.010696,0.046701,0.046624,0.229399,0.794663
10104,0.014750,0.013355,0.071858,0.071684,0.186297,0.645353
10107,0.021769,0.020374,0.063172,0.063197,0.322395,1.116808
10318,0.010662,0.009267,0.074134,0.074131,0.125005,0.433031
10333,0.017095,0.015700,0.214573,0.214508,0.073189,0.253535
12079,0.010284,0.008889,0.096507,0.096479,0.092136,0.319169


In [13]:
sharpe_ratio_annual = return_stats[['sharpe_ratio_ANN']].reset_index()
print("Table B4:")
print(sharpe_ratio_annual)

Table B4:
   permno  sharpe_ratio_ANN
0   10026          0.264554
1   10032          0.573583
2   10044         -0.254001
3   10065          0.794663
4   10104          0.645353
5   10107          1.116808
6   10318          0.433031
7   10333          0.253535
8   12079          0.319169
9   21020         -0.093799


Question 5

In [18]:
portfolio_excess_returns = crsp_factors.groupby('yearmonth')['retex'].mean()
portfolio_excess_returns = portfolio_excess_returns.reset_index()
portfolio_excess_returns.columns = ['yearmonth', 'portfolio_excess_return']
print(portfolio_excess_returns.head())

portfolio_excess_returns.to_csv("portfolio_excess_returns.csv", index=False)

mean_portfolio_excess_return = portfolio_excess_returns['portfolio_excess_return'].mean()
print(f"Mean monthly excess return: {mean_portfolio_excess_return}")

  yearmonth  portfolio_excess_return
0   2015-01                -0.072080
1   2015-02                 0.069594
2   2015-03                -0.007718
3   2015-04                -0.021759
4   2015-05                 0.026515
Mean monthly excess return: 0.008480515833333332
